In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer


In [2]:
pd.__version__

'1.4.2'

In [3]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_test = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [4]:
import sklearn

In [5]:
sklearn.__version__

'1.0.2'

In [6]:
print(len(df.columns))

19


In [7]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [8]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [9]:
std_duration = df['duration'].std()
print(std_duration)

42.594351241920904


In [10]:
initial_count = len(df)

df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

filtered_count = len(df)

print((filtered_count / initial_count) * 100)

98.1220282212598


In [11]:
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

In [12]:
dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')


In [13]:
dv = DictVectorizer()
X = dv.fit_transform(dicts)

In [14]:
X.shape

(3009173, 515)

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [16]:
y = df['duration'].values

In [17]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [18]:
y_pred = model.predict(X)

In [19]:
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(rmse)

7.649261027792376


In [20]:
df_test['duration'] = (df_test['tpep_dropoff_datetime'] - df_test['tpep_pickup_datetime']).dt.total_seconds() / 60
df_test = df_test[(df_test['duration'] >= 1) & (df_test['duration'] <= 60)].copy()


In [21]:
df_test['PULocationID'] = df_test['PULocationID'].astype(str)
df_test['DOLocationID'] = df_test['DOLocationID'].astype(str)

In [22]:
val_dicts = df_test[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_val = dv.transform(val_dicts) 

In [23]:
y_val = df_test['duration'].values
y_pred = model.predict(X_val)

from sklearn.metrics import mean_squared_error
import numpy as np

rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))

In [24]:
print(rmse_val)

7.811832836304415
